In [1]:
# Import modules

import requests
import os
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy.oauth2 import SpotifyOAuth
import spotipy.util as util
from env import client_id, client_secret, redirect_uri

In [2]:
# Define Spotify environment
os.environ['SPOTIPY_CLIENT_ID'] = client_id
os.environ['SPOTIPY_CLIENT_SECRET'] = client_secret
os.environ['SPOTIPY_REDIRECT_URI'] = redirect_uri

In [3]:
# What I'm looking for
scope = 'user-top-read'
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(scope=scope))

# Retrieve 30 Day Top
short_term_tracks = sp.current_user_top_tracks(limit=50,offset=0,time_range='short_term')
# Write to JSON
with open('short_term_top_tracks.json','w',encoding='utf-8') as f:
    json.dump(short_term_tracks,f, ensure_ascii = False, indent = 4)

# Retrieve 6 Month Top
medium_term_tracks = sp.current_user_top_tracks(limit=50, offset = 0, time_range = 'medium_term')
# Write to JSON
with open('medium_term_top_tracks.json','w',encoding='utf-8') as f:
    json.dump(medium_term_tracks,f, ensure_ascii = False, indent = 4)

# Retrieve Years Top
long_term_tracks = sp.current_user_top_tracks(limit=50, offset = 0, time_range = 'long_term')
# Write to JSON
with open('long_term_top_tracks.json','w',encoding='utf-8') as f:
    json.dump(long_term_tracks,f, ensure_ascii = False, indent = 4)

In [4]:
# Open all JSON
with open('short_term_top_tracks.json') as f:
    short_data = json.load(f)
with open('medium_term_top_tracks.json') as f:
    medium_data = json.load(f)
with open('long_term_top_tracks.json') as f:
    long_data = json.load(f)

### Create dataframes for each term of data

In [5]:
artist_name = []
song_name = []
song_uri = []
album_name = []
for track in short_data['items']:
    track['album']
    artist_name.append(track['artists'][0]['name'])
    song_name.append(track['name'])
    song_uri.append(track['uri'])
    album_name.append(track['album']['name'])

short_df = pd.DataFrame({'artist':artist_name,
'song':song_name,
'album':album_name,
'song_uri':song_uri,
'term':'short'})

In [6]:
artist_name = []
song_name = []
song_uri = []
album_name = []
for track in medium_data['items']:
    track['album']
    artist_name.append(track['artists'][0]['name'])
    song_name.append(track['name'])
    song_uri.append(track['uri'])
    album_name.append(track['album']['name'])

medium_df = pd.DataFrame({'artist':artist_name,
'song':song_name,
'album':album_name,
'song_uri':song_uri,
'term':'medium'})

In [7]:
artist_name = []
song_name = []
song_uri = []
album_name = []
for track in long_data['items']:
    track['album']
    artist_name.append(track['artists'][0]['name'])
    song_name.append(track['name'])
    song_uri.append(track['uri'])
    album_name.append(track['album']['name'])

long_df = pd.DataFrame({'artist':artist_name,
'song':song_name,
'album':album_name,
'song_uri':song_uri,
'term':'long'})

In [8]:
print(short_df.shape)
print(medium_df.shape)
print(long_df.shape)

(50, 5)
(50, 5)
(50, 5)


In [9]:
# Combine all three dataframes into one top songs dataframe
dfs = [short_df, medium_df, long_df]
df = pd.concat(dfs)

In [10]:
df.shape

(150, 5)

In [11]:
df.shape

(150, 5)

In [12]:
first_100 = pd.DataFrame(sp.audio_features(tracks = df.song_uri.head(100)))

In [13]:
first_100.shape

(100, 18)

In [14]:
last_50 = pd.DataFrame(sp.audio_features(tracks= df.song_uri.tail(50)))

In [15]:
last_50.shape

(50, 18)

In [16]:
top_100_df = pd.merge(df,first_100,how='right',left_on='song_uri',right_on='uri')
top_50_df = pd.merge(df,last_50,how='right',left_on='song_uri',right_on='uri')

In [17]:
df = pd.concat([top_100_df, top_50_df])

In [18]:
df.shape

(200, 23)

In [19]:
df.drop_duplicates(inplace=True)

In [20]:
df.shape

(150, 23)

In [23]:
df.artist.value_counts()

Delta Sleep                24
Minus the Bear             15
Leon Bridges                7
Good Kid                    6
Origami Angel               6
                           ..
Bear's Den                  1
Chinese Football            1
Dear and the Headlights     1
Dance à la Plage            1
Sleepy Dog                  1
Name: artist, Length: 68, dtype: int64